In [3]:
from bs4 import BeautifulSoup
import os
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time 

In [4]:
SEASONS = list(range(2022, 2023))

In [5]:
DATA_DIR = 'data'
STANDINGS_DIR = os.path.join(DATA_DIR, 'standings')
SCORES_DIR = os.path.join(DATA_DIR, 'scores')

In [6]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [15]:
async def scrape_season(season):
    url = f'https://www.basketball-reference.com/leagues/NBA_{season}_standings.html'
    html = await get_html(url, '#content .filter')
    soup = BeautifulSoup(html)
    links = soup.find_all('a')
    href = [l["href"] for l in links]
    standings_pages = [f"https://www.basketball-reference.com{h}" for h in href]  
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split('/')[-1])
        if os.path.exists(save_path):    
            continue
        html = await get_html(url, '#all_schedule')
        with open(save_path, 'w+') as f:
            f.write(html)

In [11]:
season = 2023
url = f'https://www.basketball-reference.com/leagues/NBA_{season}_games.html'

In [12]:
html = await get_html(url, '#content .filter')

2022-23 NBA Schedule | Basketball-Reference.com


In [13]:
soup = BeautifulSoup(html)
links = soup.find_all('a')
href = [l["href"] for l in links]
standings_pages = [f"https://www.basketball-reference.com{h}" for h in href]

In [14]:
standings_pages

['https://www.basketball-reference.com/leagues/NBA_2023_games-october.html',
 'https://www.basketball-reference.com/leagues/NBA_2023_games-november.html',
 'https://www.basketball-reference.com/leagues/NBA_2023_games-december.html',
 'https://www.basketball-reference.com/leagues/NBA_2023_games-january.html',
 'https://www.basketball-reference.com/leagues/NBA_2023_games-february.html',
 'https://www.basketball-reference.com/leagues/NBA_2023_games-march.html',
 'https://www.basketball-reference.com/leagues/NBA_2023_games-april.html',
 'https://www.basketball-reference.com/leagues/NBA_2023_games-may.html',
 'https://www.basketball-reference.com/leagues/NBA_2023_games-june.html']